In [ ]:
# Install deepinv (skip if already installed)
%pip install deepinv

<!-- MathJax macro definitions inserted automatically -->
$$
\newcommand{\forw}[1]{{A\left({#1}\right)}}
\newcommand{\noise}[1]{{N\left({#1}\right)}}
\newcommand{\inverse}[1]{{R\left({#1}\right)}}
\newcommand{\inversef}[2]{{R\left({#1},{#2}\right)}}
\newcommand{\inversename}{R}
\newcommand{\reg}[1]{{g_\sigma\left({#1}\right)}}
\newcommand{\regname}{g_\sigma}
\newcommand{\sensor}[1]{{\eta\left({#1}\right)}}
\newcommand{\datafid}[2]{{f\left({#1},{#2}\right)}}
\newcommand{\datafidname}{f}
\newcommand{\distance}[2]{{d\left({#1},{#2}\right)}}
\newcommand{\distancename}{d}
\newcommand{\denoiser}[2]{{\operatorname{D}_{{#2}}\left({#1}\right)}}
\newcommand{\denoisername}{\operatorname{D}_{\sigma}}
\newcommand{\xset}{\mathcal{X}}
\newcommand{\yset}{\mathcal{Y}}
\newcommand{\group}{\mathcal{G}}
\newcommand{\metric}[2]{{d\left({#1},{#2}\right)}}
\newcommand{\loss}[1]{{\mathcal\left({#1}\right)}}
\newcommand{\conj}[1]{{\overline{#1}^{\top}}}
$$


# Training a reconstruction model

This example provides a very simple quick start introduction to training reconstruction networks with
DeepInverse for solving imaging inverse problems.

Training requires these components, all of which you can define with DeepInverse:

* A `model` to be trained from reconstructors or define your own.
* A `physics` from our list of physics. Or, [bring your own physics](https://deepinv.github.io/deepinv/auto_examples/basics/demo_custom_dataset.html#sphx-glr-auto-examples-basics-demo-custom-dataset-py).
* A `dataset` of images and/or measurements from datasets. Or, [bring your own dataset](https://deepinv.github.io/deepinv/auto_examples/basics/demo_custom_dataset.html#sphx-glr-auto-examples-basics-demo-custom-dataset-py).
* A `loss` from our loss functions.
* A `metric` from our metrics.

Here, we demonstrate a simple experiment of training a UNet
on an inpainting task on the Urban100 dataset of natural images.

In [ ]:
import deepinv as dinv
import torch

device = dinv.utils.get_freer_gpu() if torch.cuda.is_available() else "cpu"
rng = torch.Generator(device=device).manual_seed(0)

## Setup

First, define the physics that we want to train on.



In [ ]:
physics = dinv.physics.Inpainting((1, 64, 64), mask=0.8, device=device, rng=rng)

Then define the dataset. Here we simulate a dataset of measurements from Urban100.

> **Tip**
>
> See datasets for types of datasets DeepInverse supports: e.g. paired, ground-truth-free,
> single-image...
>
>


In [ ]:
from torchvision.transforms import Compose, ToTensor, Resize, CenterCrop, Grayscale

dataset = dinv.datasets.Urban100HR(
    ".",
    download=True,
    transform=Compose([ToTensor(), Grayscale(), Resize(256), CenterCrop(64)]),
)

train_dataset, test_dataset = torch.utils.data.random_split(
    torch.utils.data.Subset(dataset, range(50)), (0.8, 0.2)
)

dataset_path = dinv.datasets.generate_dataset(
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    physics=physics,
    device=device,
    save_dir=".",
    batch_size=1,
)

train_dataloader = torch.utils.data.DataLoader(
    dinv.datasets.HDF5Dataset(dataset_path, train=True), shuffle=True
)
test_dataloader = torch.utils.data.DataLoader(
    dinv.datasets.HDF5Dataset(dataset_path, train=False), shuffle=False
)

Visualize a data sample:



In [ ]:
x, y = next(iter(test_dataloader))
dinv.utils.plot({"Ground truth": x, "Measurement": y, "Mask": physics.mask})

For the model we use an artifact removal model, where
$\phi_{\theta}$ is a U-Net

\begin{align}f_{\theta}(y) = \phi_{\theta}(A^{\top}(y))\end{align}



In [ ]:
model = dinv.models.ArtifactRemoval(
    dinv.models.UNet(1, 1, scales=2, batch_norm=False).to(device)
)

## Train the model
We train the model using the [`deepinv.Trainer`](https://deepinv.github.io/deepinv/api/stubs/deepinv.Trainer.html) class,
which cleanly handles all steps for training.

We perform supervised learning and use the mean squared error as loss function.
See losses for all supported state-of-the-art loss functions.

We evaluate using the PSNR metric.
See metrics for all supported metrics.

<div class="alert alert-info"><h4>Note</h4><p>In this example, we only train for a few epochs to keep the training time short.
      For a good reconstruction quality, we recommend to train for at least 100 epochs.</p></div>



In [ ]:
trainer = dinv.Trainer(
    model=model,
    physics=physics,
    optimizer=torch.optim.Adam(model.parameters(), lr=1e-3),
    train_dataloader=train_dataloader,
    eval_dataloader=test_dataloader,
    epochs=5,
    losses=dinv.loss.SupLoss(metric=dinv.metric.MSE()),
    metrics=dinv.metric.PSNR(),
    device=device,
    plot_images=True,
    show_progress_bar=False,
)

_ = trainer.train()

## Test the network
We can now test the trained network using the [`deepinv.test`](https://deepinv.github.io/deepinv/api/stubs/deepinv.test.html) function.

The testing function will compute metrics and plot and save the results.


In [ ]:
trainer.test(test_dataloader)